In [1]:
import pickle
from pathlib import Path
import pandas as pd 
import os
import sys
import matplotlib.pyplot as plt
import getpass

In [2]:
#add the github path that will run the notebooks: 
username = getpass.getuser() # gets the username from the current comp 
print (username)

gilmandelbaum


In [3]:
sys.path.insert(0,'/Users/{:}/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot'.format(username))

In [4]:
sys.path[0]

'/Users/gilmandelbaum/github/analysis-pipeline-for-photometry_ex/Nb_data_set/1.data_set_lineplot'

In [5]:
import data_set_lineplot_ic_together_helper_functions as dslhf

In [6]:
sys.path.append('/Users/{}/GitHub/CaImAn/caiman/source_extraction/cnmf/'.format(username))
import deconvolution

In [7]:
seq_str = '0a1b2a3a4abcd5a6a7a'
period_of_interest = "choice_lick_label_50_sec"
HowManyBasck=1
folderName="right_dual_color_photo_full_data_after_QC" #folder where the plot data_set will be saved 
path = "/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis"

# Folder name of analysis: 

In [8]:
analysis_type ="full_session_traces_deconvalution"

In [9]:
sys.executable

'/anaconda3/envs/caiman/bin/python'

# Import the data and where to save it 

In [10]:
path_to_save = path+"/"+folderName+"/"+seq_str #also for loading

In [11]:
path_to_save

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis/right_dual_color_photo_full_data_after_QC/0a1b2a3a4abcd5a6a7a'

# Import the data table and data

In [12]:
MasterSheetName = "3.lateral_ medial_6_lateral_channels_final"
Mouse_Date_FileName = pd.read_excel(path+'/'+MasterSheetName+'.xlsx')
Mouse_Date_FileName

mouse    date    file name
0   T163  181207  T163-123432
1   T163  181128  T163-115220
2   T163  181203  T163-115901
3   T163  181130  T163-133215
4   T163  181204  T163-114628
..   ...     ...          ...
70  T202  190530  T202-152554
71  T202  190522  T202-141516
72  T202  190529  T202-160644
73  T202  190526  T202-142728
74  T202  190516  T202-165613

[75 rows x 3 columns]

In [13]:
%%time
root = Path(path_to_save)
d = "data_object_photometry_after_down_sampling"+"_"+seq_str+'.pickle'
my_path = root / d 
fileToOpen = open(my_path, 'rb')
data_set = pickle.load(fileToOpen)

CPU times: user 679 ms, sys: 1.25 s, total: 1.93 s
Wall time: 4.04 s


# make a list to add to plots 

In [14]:
Mouse_Date_FileName['date'] = Mouse_Date_FileName['date'].apply(str)
Mouse_Date_FileName['mouse_session'] = Mouse_Date_FileName[['mouse','date']].apply(lambda x: '-'.join(x), axis=1)
session_date= list(Mouse_Date_FileName['mouse_session']) #session list

# Make the y axis label 

In [15]:
y_axis_list = ["df_f","z_score","z_score_sliding_window"]

In [16]:
alphabet = ["a","b","c","d","e","f","g"]
y_axis = y_axis_list[alphabet.index(seq_str[:seq_str.index('4')][-1])]
print (y_axis)

df_f


In [17]:
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(path_to_save,analysis_type,period_of_interest)
if not os.path.exists(cache_dir_plt):
    os.makedirs(cache_dir_plt)

In [18]:
cache_dir_plt

'/Volumes/GilInVivoPaper.large/2.Photometry_in_str/1.lateral_medial_6_analysis/right_dual_color_photo_full_data_after_QC/0a1b2a3a4abcd5a6a7a/full_session_traces_deconvalution/choice_lick_label_50_sec'

In [19]:
keys = ["d2 R","d1 R","d2 L","d1 L"]

In [20]:
# state_Selection =[(39, 47), (47, 48), (48, 48), (48, 49)]
state_Selection=[(48, 50)]

In [21]:
def plot_cue_photo_per_session(data_set_l,session_date_l,states_l):
    
    for data,date in zip(data_set_l,session_date_l):
        
        path_to_plot=cache_dir_plt+"/"+str(date)+"_"+seq_str[:seq_str.index("4")]+"_plot"
        print(date+" is being plotted")

        time_stamp =find_time_stamps(data,states_l)
        
        plot_time_stamp_photo(data,time_stamp,path_to_plot)

        
    return ()

In [ ]:
def find_time_stamps(data,states_list):
    time_bin_list=[]
    
    for state in states_list:
        state_start=state[0]
        state_end=state[1]
        rows = data[(data['iState_start']==state_start) & (data['iState_end']==state_end)]
        indexes = list(rows.index)
        time_bin_list.extend(indexes)

    return time_bin_list



In [ ]:
def plot_time_stamp_photo(data,time_stamp, path_to_plot):
    fig, ax = plt.subplots(4, 1,figsize=(10,10),sharex=True, sharey=False)
    
    for _ax, k in zip(ax.flatten(), keys):
        _ax.plot(list(range(len(data[k]))), data[k])
        _ax.set_title(k.split('_')[0])
        _ax.axhline(y=-1,color='r',linewidth=0.5)
        _ax.axhline(y=2,color='r',linewidth=0.5)
        _ax.set_ylim(-2.5,4)
        _ax.set_xlim(100000,110000)
        
        for time in time_stamp:
            _ax.axvline(x=time,linewidth=1, color='g',linestyle="--")
        
        
    _ax.set_xlabel('Time(ms)')
    _ax.set_ylabel("Zscore")
   
    fig.savefig(path_to_plot+'.pdf')
    plt.show()

In [ ]:
%%time
plot_cue_photo_per_session(data_set,session_date,state_Selection)